In [1]:
import numpy as np
import random

In [12]:
layers=[4,3,2] #specify the layer sizes here 


#bala: added options for having different activations for different layers
global activation_codes 
activation_codes= []  
for ind in range(len(layers)-1):
    #print('ind: %d' %(ind))
    #activation_codes.append("linear") 
    #activation_codes.append("sigmoid") 
    #you can modify the activation function here depending upon your needs
    if ind==0:
        activation_codes.append("sigmoid") #first and second are sigmoid 
    else:
        #activation_codes.append("linear")
        activation_codes.append("softmax")
        
    #if ind==len(layers)-2:
    #    activation_codes.append("linear") #the last layer has linear activation 
    #else:
    #    activation_codes.append("sigmoid") #all other layers have sigmoid activation

print(activation_codes)


global cost_code 
#cost_code = "mse" #you can use cross_entropy, hinge, etc. 
cost_code="cross"
global num_layers 
num_layers=len(layers)


eta=0.001 #learning rate
epochs=30 #num epochs
mini_batch_size=1 #currently mini batch size is just one



def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    sigm=sigmoid(z)
    return sigm*(1-sigm)

def linear_prime(z):
    return (z-z+1)

def relu_prime(z):
    zz = [1 if ele>0 else 0 for ele in z]
    return (zz.transpose)

def soft_max(z):
    p= np.exp(z) / sum(np.exp(z))
    return(p)

def activation_prime(z,activation_fn):
    if activation_fn=='sigmoid':
        deriv = sigmoid_prime(z)
    elif activation_fn=='linear':
        deriv= linear_prime(z)
    elif activation_fn=='relu':
        deriv= relu_prime(z)
    #elif activation_fn=='tanh': #need to add this later
        #x=tanh(0,np.dot(w,x)+b)
    else:
        print('Unknown activation function:%s Please check !' %(activation_fn))
        deriv = soft_max(z)
    return deriv

def cost_derivative(output_activation,y,cost_code):
    y=np.reshape(y,(-1,1))
    if cost_code=='mse':
        deriv = (output_activation-y)
    elif cost_code=='cross':
        #print('cost code not yet implemented. using default cost mse')
        deriv = (output_activation-y)
    return deriv 

def feed_forward_compute_activation(b,w,x,activation_fn):
    x=np.dot(w,x)+b
    if activation_fn=='sigmoid':
        x_act = sigmoid(x)
    elif activation_fn=='linear':
        x_act=x
    elif activation_fn=='relu':
        #print(x.shape)
        #print(x)
        x_act=np.max(0,x)
    #elif activation_fn=='tanh': #need to add this later
        #x=tanh(0,np.dot(w,x)+b)
    else:
        #print('Unknown activation function:%s Please check !' %(activation_fn))
        print("softmax")
        x_act=soft_max(x) #call sigmoid by default 
    return x, x_act

def feed_forward(biases,weights,x):
    activation = x
    activations = [x]
    zs = []
    layer = 0
    for b, w in zip(biases,weights):
        z, activation = feed_forward_compute_activation(b,w,activation,activation_codes[layer])
        #z = np.dot(w,activation)+b
        zs.append(z)
        #activation = sigmoid(z)
        activations.append(activation)
        layer = layer+1
    return zs, activations 

def feed_forward_eval(biases,weights,x):
    activation = x
    layer = 0
    for b, w in zip(biases,weights):
        z, activation = feed_forward_compute_activation(b,w,activation,activation_codes[layer])
        layer = layer + 1
    return activation

def backprop(biases,weights,x,y,zs,activations):
    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]
    #separate handling of last layer
    layer = -1
    #print(activations[layer],y)
    #delta = cost_derivative(activations[layer],y,cost_code)*activation_prime(zs[layer], activation_codes[layer]) # sigmoid_prime()
    delta = cost_derivative(activations[layer],y,cost_code)
    nabla_b[-1] = delta
    nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    for l in range(2, num_layers):
        sp = activation_prime(zs[-l], activation_codes[-l])
        delta = np.dot(weights[-l+1].transpose(),delta) * sp
        nabla_b[-l] = delta
        nabla_w[-l] = np.dot(delta,activations[-l-1].transpose())
    return (nabla_b, nabla_w)


def update_mini_batch_sgd(biases,weights,mini_batch,eta):
    nabla_b=[np.zeros(b.shape) for b in biases]
    nabla_w=[np.zeros(w.shape) for w in weights]
    for x,y in mini_batch:
        print("x1")
        print(x)
        zs, activations = feed_forward(biases,weights,x)
        print("backPROP")
        delta_nabla_b,delta_nabla_w=backprop(biases,weights,x,y,zs, activations)
        nabla_b=[nb+dnb for nb,dnb in zip(nabla_b,delta_nabla_b)]
        nabla_w=[nw+dnw for nw,dnw in zip(nabla_w,delta_nabla_w)]
        
    weights=[w-(eta/(len(mini_batch)))*nw for w,nw in zip(weights,nabla_w)]
    biases=[b-(eta/len(mini_batch))*nb for b,nb in zip(biases,nabla_b)]
    #print(weights)
    return biases,weights


def evaluate(biases,weights,test_data):
    test_results=[(np.argmax(feed_forward_eval(biases,weights,x)),y) for x,y in test_data]
    return sum(int(x==y) for x,y in test_results)

def evaluate_regression(biases,weights,test_data):
    #for x,y in test_data:
    #    print(feed_forward_eval(biases,weights,x))
    #test_results=[np.asscalar(feed_forward_eval(biases,weights,x))-y for x,y in test_data]
    #test_results=[(feed_forward_eval(biases,weights,x))-y for x,y in test_data]
    test_results=[]
    for x,y in test_data:
        y=np.reshape(y,(-1,1))
        a=(feed_forward_eval(biases,weights,x))
        soft= np.exp(a) / sum(np.exp(a))
        print("soft")
        print(soft)
        print("cross")
        cross=-(np.sum(y*np.log(soft)))
        print(cross)
        test_results.append(cross)
    print("test results")
    print(test_results)
    #print("soft")
    #soft=np.zeros((layers[-1],1),dtype=float)
    #soft= np.exp(test_results[0]) / sum(np.exp(test_results[0]))
    #print(soft)
    #cross=-(np.sum(y*np.log(predictions+1e-9)))
    #test_results=[np.asscalar(feed_forward_eval(biases,weights,x)) for x,y in test_data]
    #print(np.dot(test_results,test_results))
    #return 0.5*(np.dot(test_results,test_results))
    #return 0.5*sum((x-y)*(x-y) for x,y in test_results)
    return test_results


def sgd(layers,epochs):
    random.seed(1000)
    np.random.seed(1000)
    
    biases=[np.random.randn(y,1) for y in layers[1:]]
    weights=[np.random.randn(y,x) for x,y in zip(layers[:-1],layers[1:])]
    
    #train_data1=list(train_data)
    #n=len(train_data1)
    #test_data1=list(test_data)
    #accuracies=[]
    regression_costs= []
    mini_batch_size = 1
    k=1
    total_regression_cost=[0]
    for j in range(epochs):
        a=0  
        print('training ...')
        a=a+1
        train_x=np.zeros((4,1),dtype=float)
        train_x=[1,2,3,4]
        train_x = [np.resize(train_x,(4,1))]
        y_true_batch=[1,0]
        train_y = np.reshape(y_true_batch,(1,-1))

        training_data = zip(train_x, train_y)

        mini_batch = list(training_data)
        #print(mini_batch)
            
        eta = 1.0/a
        #update biases and weights
        biases,weights=update_mini_batch_sgd(biases,weights,mini_batch,eta)
        if(a%10==0):
            print('*')
            
            
        a=0
        #testing part
        print('testing ...')
        #total_regression_cost = 0
        
        a=a+1
        #train_y = np.reshape(y_true_batch,(1,-1))

        training_data = zip(train_x, train_y)

        mini_batch = list(training_data)
        #print(mini_batch)

        regression_costs = evaluate_regression(biases,weights,mini_batch)
        total_regression_cost+=(regression_costs)
        print("total regression")
        print(total_regression_cost)

        #eta = 1.0/a
        #update biases and weights
        #biases,weights=update_mini_batch_sgd(biases,weights,mini_batch,eta)
        #if(a%10==0):
        #    print('*')

        print('regression cost: %f' %(sum(total_regression_cost)/a))
        print('********************')
sgd(layers,1)

['sigmoid', 'softmax']
training ...
x1
[[1]
 [2]
 [3]
 [4]]
softmax
backPROP
testing ...
softmax
soft
[[ 0.70109693]
 [ 0.29890307]]
cross
0.355109125206
test results
[0.35510912520646509]
total regression
[0, 0.35510912520646509]
regression cost: 0.355109
********************
